<a href="https://colab.research.google.com/github/pavanarya/AI-ML/blob/main/ML/Linear_Regression/1.Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Step1:
Processing Input Data, In this case we will simply generate input data using numpy